In [ ]:
!pip install rouge
!pip install farasapy
!git clone https://github.com/aub-mind/arabert
!pip install pyarabic
!pip install datasets
!pip install transformers
!pip install wandb

In [ ]:
!wget "https://raw.githubusercontent.com/SalehShmali/Arabic_News_Summarization/main/articles.csv"

In [6]:
from rouge import Rouge
import pandas as pd
from arabert.preprocess import ArabertPreprocessor
from datasets import load_dataset
from transformers import BertTokenizerFast,GPT2TokenizerFast, EncoderDecoderModel
from transformers import Seq2SeqTrainingArguments,Seq2SeqTrainer

In [5]:
# preview some data
df = pd.read_csv('articles.csv')
data=df[['title','content']]
data.head(5)

,title,content
0,الجيش التركي يعلن جاهزيته للعبور إلى ليبيا.. ا...,تجددت المعارك اليوم الجمعة بمحيط العاصمة الليب...
1,"محمد علي يعلن وثيقة ""التوافق المصري"" تمهيدا لإ...",عبد الرحمن محمد-الجزيرة نت كشف الفنان والمقاول...
2,من الحلم إلى الشارع.. قصة مشروع السيارة التركية,خليل مبروك-إسطنبول شهدت تركيا اليوم الجمعة الإ...
3,حكومة الهند تصعّد ضد المسلمين.. طوارئ بنيودلهي...,عززت السلطات الهندية اليوم الجمعة إجراءات الأم...
4,مظاهرات بأربع دول إسلامية ضد انتهاكات الصين بح...,احتشد آلاف المسلمين في عدد من الدول الإسلامية ...


In [16]:
rouge = Rouge()
batch_size = 4
encoder_max_length=512
decoder_max_length=128
arabert="aubmindlab/bert-base-arabert"
aragpt2 = "aubmindlab/aragpt2-base"
arabert_prep = ArabertPreprocessor(model_name=arabert, keep_emojis=False)

[2022-04-08 10:43:19,021 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [17]:
# preview processing for arabic text
sentence= data['content'][100]
processed_sentence= arabert_prep.preprocess(sentence)
processed_sentence

'أعلن وزير ال+ بترول و+ ال+ ثرو +ة ال+ معدني +ة ال+ مصري +ة طارق ال+ ملا ال+ يوم عن إرساء ثلاث +ة قطاع +ات ل+ ال+ تنقيب عن ال+ نفط و+ ال+ غاز في ال+ بحر ال+ أحمر . و+ قال ال+ ملا إن شرك +ة شيفرون ال+ أميركي +ة فاز +ت في ال+ مزايد +ة ب+ أحد ال+ قطاع +ات . +كما فاز +ت مجموع +ة رويال داتش شل ال+ بريطاني +ة ال+ هولندي +ة ب+ قطاع ثاني ، في حين كان ال+ قطاع ال+ ثالث من نصيب تحالف شركتي شل و+ مبادل +ة ال+ إماراتي +ة . و+ أضاف ال+ وزير ال+ مصري أن إجمالي مساح +ة ال+ قطاع +ات ال+ ثلاث +ة تبلغ عشر +ة آلاف كيلومتر مربع ، و+ أشار إلى أن ال+ حد ال+ أدنى ل+ ال+ استثمار +ات يبلغ 326 مليون دولار ، ب+ حيث ترتفع إلى مليار +ات عد +ة من ال+ دولار +ات في مراحل ال+ تنمي +ة في حال تحقيق ال+ اكتشاف +ات . ال+ وزير أشار أيض +ا إلى أن بلاد +ه وقع +ت 99 اتفاقي +ة بترولي +ة خلال ال+ سنو +ات ال+ خمس ال+ ماضي +ة ب+ حد أدنى ل+ ال+ استثمار +ات يبلغ حوالي 16 مليار دولار . و+ أوضح وزير ال+ بترول أن من أهم نجاح +ات قطاع ال+ طاق +ة تنفيذ 27 مشروع +ا ل+ تنمي +ة حقول ال+ غاز ب+ استثمار +ات تقدر ب+ 31 مليار دولار ، و+ وصول إ

In [18]:
bert_tokenizer = BertTokenizerFast.from_pretrained(arabert)
bert_tokenizer.bos_token = bert_tokenizer.cls_token
bert_tokenizer.eos_token = bert_tokenizer.sep_token

In [19]:
tokenized_data = bert_tokenizer(processed_sentence, padding="max_length", truncation=True, max_length=encoder_max_length)
tokenized_data

{'input_ids': [29756, 11242, 23721, 3000, 32415, 897, 3000, 3620, 125, 3000, 39651, 125, 3000, 21731, 125, 18737, 3000, 6429, 3000, 7447, 745, 30837, 15446, 125, 20122, 1012, 816, 3000, 34375, 745, 3000, 6748, 897, 3000, 5520, 781, 3000, 3092, 3000, 10793, 11, 897, 5790, 3000, 6429, 425, 4916, 125, 50024, 3000, 46885, 125, 5640, 126, 781, 3000, 39235, 125, 448, 2475, 3000, 20122, 1012, 11, 8270, 5640, 126, 38874, 125, 35732, 16503, 669, 3000, 55343, 125, 3000, 52071, 125, 448, 20122, 15423, 81, 781, 3995, 5951, 3000, 20122, 3000, 15419, 857, 22984, 33279, 36474, 669, 897, 38448, 125, 3000, 54791, 125, 11, 897, 11161, 3000, 23721, 3000, 21731, 401, 47035, 21546, 125, 3000, 20122, 1012, 3000, 15446, 125, 13720, 5412, 125, 10575, 56358, 21419, 81, 897, 11093, 2781, 401, 3000, 523, 3000, 10881, 816, 3000, 54909, 1012, 23982, 1392, 39993, 35428, 81, 448, 3986, 33485, 2781, 39987, 1012, 730, 125, 857, 3000, 35428, 1012, 781, 39106, 3000, 15164, 125, 781, 3823, 33336, 3000, 47781, 1012, 11, 3

In [ ]:
def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
    outputs = [self.bos_token_id] + token_ids_0 + [self.eos_token_id]
    return outputs

In [ ]:
GPT2TokenizerFast.build_inputs_with_special_tokens = build_inputs_with_special_tokens
gpt2_tokenizer = GPT2TokenizerFast.from_pretrained(aragpt2)
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token


In [ ]:
def process_data_to_model_inputs(batch):                                                                                                           
    inputs = bert_tokenizer(batch["content"], padding="max_length", truncation=True, max_length=encoder_max_length)
    outputs = gpt2_tokenizer(batch["title"], padding="max_length", truncation=True, max_length=decoder_max_length)
                                                                                                        
    batch["input_ids"] = inputs.input_ids                                                              
    batch["attention_mask"] = inputs.attention_mask                                                     
    batch["decoder_input_ids"] = outputs.input_ids                                                      
    batch["labels"] = outputs.input_ids.copy() 
    batch["decoder_attention_mask"] = outputs.attention_mask
    # mask loss for padding                                                                             
    batch["labels"] = [
        [-100 if mask == 0 else token for mask, token in mask_and_tokens] for mask_and_tokens in [zip(masks, labels) for masks, labels in zip(batch["decoder_attention_mask"], batch["labels"])]
    ]

    assert all([len(x) == encoder_max_length for x in inputs.input_ids])
    assert all([len(x) == decoder_max_length for x in outputs.input_ids])
    return batch

In [ ]:
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = gpt2_tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = gpt2_tokenizer.eos_token_id
    label_str = gpt2_tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.get_scores(pred_str, label_str, avg=True)

    return {
        "rouge2_precision": round(rouge_output["rouge-2"]["p"], 4),
        "rouge2_recall": round(rouge_output["rouge-2"]["r"], 4),
        "rouge2_fmeasure": round(rouge_output["rouge-2"]["f"], 4),
    }

In [ ]:
all_data = load_dataset("ArabicNewsSummary.py")
train_data = all_data['train'].train_test_split(test_size=0.1,seed=42)['train']
val_data = all_data['train'].train_test_split(test_size=0.1,seed=42)['test']

In [ ]:
print("Length of train data",len(train_data))
print("Length of val data",len(val_data))

In [ ]:
# make train dataset ready
train_data = train_data.map(
    process_data_to_model_inputs, batched=True, batch_size=batch_size, remove_columns=["content", "title"],
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

# same for validation dataset
val_data = val_data.map(
    process_data_to_model_inputs, batched=True, batch_size=batch_size, remove_columns=["content", "title"],
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [ ]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained(arabert, aragpt2)

In [ ]:
model.config.decoder_start_token_id = gpt2_tokenizer.bos_token_id
model.config.pad_token_id = gpt2_tokenizer.eos_token_id
model.config.max_length = 128
model.config.min_length = 64
model.config.no_repeat_ngram_size = 3
model.early_stopping = True
model.length_penalty = 2.0
model.num_beams = 4

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./model",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size//2,
    predict_with_generate=True,
    do_eval=True,
    evaluation_strategy ="epoch",
    do_train=True,
    logging_steps=3963 //3,  
    save_steps= 3963 //3,  
    warmup_steps=1000,
    eval_steps=10,
    num_train_epochs=3,
    overwrite_output_dir=True,
    save_total_limit=10,
    fp16=True
)
    

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)

In [ ]:
trainer.train()

In [ ]:
eval_output = trainer.evaluate()